# This notebook aims to compare performance of reading/loading large file formats using Pandas and Polars
## In this notebook, I use a dataset contained 69 million rows with 7 variables from Reddid at [here](https://files.pushshift.io/reddit/69M_reddit_accounts.csv.gz)
## This csv.gz is large 1005Mb and an extracted to *CSV file is 3.3Gb.
## If you convert *csv to *parquet and/or *feather, file sizes are reduced to 893.7MB for *parquet and 805.6MB for *feather.
## The overall comparison of *csv, *parquet, and *feather formats can be summarized [here](https://twitter.com/levikul09/status/1644629913440501763/photo/1).
![alt text](https://pbs.twimg.com/media/FtLmMH4aQAA7qgk?format=png&name=medium)

### import Pandas and Polars
### Check versions of Pandas and Polars

In [3]:
import pandas as pd
import polars as pl
import time
DATA_DIR = '/users/linh/Downloads'

print(f'Pandas version: {pd.__version__}')
print(f'Polars version: {pl.__version__}')

Pandas version: 2.0.0
Polars version: 0.17.0


### Indeed, Pandas version 2 has been up-to-date using either pyarrow or NumPy engines.
### We can choose these backends via a function `dtype='pyarrow'` or a default as `dtype_backend=numpy_nullable`
`pd_feather = pd.read_feather(f"{DATA_DIR}/reddit.feather", dtype_backend='pyarrow')`

## Check how is fast to read a large file with *CSV format using Pandas and Polars
### Polars with Lazily reading

In [5]:
%time
pl_csv = (
    pl.scan_csv(f"{DATA_DIR}/reddit.csv")
 
)


CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 1.91 µs


### Polars with fully reading

In [6]:
%time
pl_csv = (
    pl.read_csv(f"{DATA_DIR}/reddit.csv")
)
pl_csv.describe()

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 2.15 µs


describe,id,name,created_utc,updated_on,comment_karma,link_karma
str,f64,str,f64,f64,f64,f64
"""count""",6.9382538e7,"""69382538""",6.9382538e7,6.9382538e7,6.9382538e7,6.9382538e7
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0
"""mean""",4.2844e9,null,1.4196e9,1.5367e9,327.284744,159.060551
"""std""",2.1789e10,null,6.5324e7,135804.527686,5143.102887,7600.103613
"""min""",1.0,"""*polhold00062""",1.1180e9,1.5365e9,-111863.0,-9108.0
"""max""",1.6450e11,"""zzzzzzzzzzzzzz…",1.5373e9,1.5374e9,1.3076606e7,2.4113433e7
"""median""",4.0664e7,null,1.4347e9,1.5367e9,0.0,0.0


In [25]:
%time
pd_csv = (
    pd.read_csv(f"{DATA_DIR}/reddit.csv") #defaults as NumPy backed
    
)

CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 178 µs


In [26]:
%time
pd_csv = (
    pd.read_csv(f"{DATA_DIR}/reddit.csv", dtype_backend='pyarrow')
    
)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 12.9 µs


In [7]:

pd_csv.describe()

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 9.06 µs


,id,created_utc,updated_on,comment_karma,link_karma
count,6.938254e+07,6.938254e+07,6.938254e+07,6.938254e+07,6.938254e+07
mean,4.284409e+09,1.419649e+09,1.536730e+09,3.272847e+02,1.590606e+02
std,2.178923e+10,6.532366e+07,1.358045e+05,5.143103e+03,7.600104e+03
min,1.000000e+00,1.118030e+09,1.536497e+09,-1.118630e+05,-9.108000e+03
25%,2.268322e+07,1.381743e+09,1.536595e+09,0.000000e+00,0.000000e+00
50%,4.066391e+07,1.434715e+09,1.536741e+09,0.000000e+00,0.000000e+00
75%,5.878004e+07,1.466821e+09,1.536861e+09,0.000000e+00,0.000000e+00
max,1.644962e+11,1.537306e+09,1.537391e+09,1.307661e+07,2.411343e+07


## This task, Pandas takes 44.7 seconds using NumPy backend and 56.9 seconds using PyArrow backend, 
## while Polars with lazily and fully reading needs 1.9 micro-seconds and 8.6 seconds, respectively.
# ==> Polars wins Pandas in the reading task using *csv format

## Check to read a large *parquet file
### Convert *csv to *parquet format, this process takes 26.7 seconds
### Now, the parquet file size is 893.7Mb

In [10]:
%time
parquet = (
    pl.scan_csv(f"{DATA_DIR}/reddit.csv")
    .sink_parquet(f"{DATA_DIR}/reddit.parquet")
)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 5.96 µs


In [11]:
%time
parquet = (
    pl.scan_parquet(f"{DATA_DIR}/reddit.parquet")
)

CPU times: user 0 ns, sys: 1e+03 ns, total: 1e+03 ns
Wall time: 8.82 µs


In [12]:
%time
pl_parquet = (
    pl.read_parquet(f"{DATA_DIR}/reddit.parquet")
)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 2.15 µs


In [13]:
pl_parquet.describe()

describe,id,name,created_utc,updated_on,comment_karma,link_karma
str,f64,str,f64,f64,f64,f64
"""count""",6.9382538e7,"""69382538""",6.9382538e7,6.9382538e7,6.9382538e7,6.9382538e7
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0
"""mean""",4.2844e9,null,1.4196e9,1.5367e9,327.284744,159.060551
"""std""",2.1789e10,null,6.5324e7,135804.527686,5143.102886,7600.103614
"""min""",1.0,"""*polhold00062""",1.1180e9,1.5365e9,-111863.0,-9108.0
"""max""",1.6450e11,"""zzzzzzzzzzzzzz…",1.5373e9,1.5374e9,1.3076606e7,2.4113433e7
"""median""",4.0664e7,null,1.4347e9,1.5367e9,0.0,0.0


In [14]:
%time
pd_parquet = (
    pd.read_parquet(f"{DATA_DIR}/reddit.parquet") # for NumPy backend
)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 5.72 µs


In [30]:
pd_feather = pd.read_parquet(f"{DATA_DIR}/reddit.parquet", dtype_backend='pyarrow')

In [16]:
pd_parquet.describe()

,id,created_utc,updated_on,comment_karma,link_karma
count,6.938254e+07,6.938254e+07,6.938254e+07,6.938254e+07,6.938254e+07
mean,4.284409e+09,1.419649e+09,1.536730e+09,3.272847e+02,1.590606e+02
std,2.178923e+10,6.532366e+07,1.358045e+05,5.143103e+03,7.600104e+03
min,1.000000e+00,1.118030e+09,1.536497e+09,-1.118630e+05,-9.108000e+03
25%,2.268322e+07,1.381743e+09,1.536595e+09,0.000000e+00,0.000000e+00
50%,4.066391e+07,1.434715e+09,1.536741e+09,0.000000e+00,0.000000e+00
75%,5.878004e+07,1.466821e+09,1.536861e+09,0.000000e+00,0.000000e+00
max,1.644962e+11,1.537306e+09,1.537391e+09,1.307661e+07,2.411343e+07


## This task, Pandas takes 4 minutes and 41.0 seconds using NumPy backend, 2.6 seconds with PyArrow backend, and 8.4 seconds to describe, 
## while Polars with lazily, fully reading and describing needs 8.82 micro-seconds, 1.5 seconds and 8.8 seconds, respectively.
# ==> Polars wins Pandas in the reading using *parquet format 

## Check to read a large *feather file
### Convert *csv to *feather format, this process takes 26.7 seconds
### Now, the parquet file size is 805.6Mb

In [31]:
%time
parquet = (
    pl.scan_csv(f"{DATA_DIR}/reddit.csv")
    .sink_ipc(f"{DATA_DIR}/reddit.feather")
)

CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 9.06 µs


In [19]:
%time
feather = (
    pl.scan_ipc(f"{DATA_DIR}/reddit.feather")
)

CPU times: user 0 ns, sys: 1e+03 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [20]:
%time
pl_feather = (
    pl.read_ipc(f"{DATA_DIR}/reddit.feather")
)

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 4.05 µs


Could not mmap compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.


In [22]:
pl_feather.describe()

describe,id,name,created_utc,updated_on,comment_karma,link_karma
str,f64,str,f64,f64,f64,f64
"""count""",6.9382538e7,"""69382538""",6.9382538e7,6.9382538e7,6.9382538e7,6.9382538e7
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0
"""mean""",4.2844e9,null,1.4196e9,1.5367e9,327.284744,159.060551
"""std""",2.1789e10,null,6.5324e7,135804.527686,5143.102887,7600.103613
"""min""",1.0,"""*polhold00062""",1.1180e9,1.5365e9,-111863.0,-9108.0
"""max""",1.6450e11,"""zzzzzzzzzzzzzz…",1.5373e9,1.5374e9,1.3076606e7,2.4113433e7
"""median""",4.0664e7,null,1.4347e9,1.5367e9,0.0,0.0


In [23]:
pd_feather = pd.read_feather(f"{DATA_DIR}/reddit.feather") # for Numpy backend

In [27]:
pd_feather = pd.read_feather(f"{DATA_DIR}/reddit.feather", dtype_backend='pyarrow')

In [24]:
pd_feather.describe()

,id,created_utc,updated_on,comment_karma,link_karma
count,6.938254e+07,6.938254e+07,6.938254e+07,6.938254e+07,6.938254e+07
mean,4.284409e+09,1.419649e+09,1.536730e+09,3.272847e+02,1.590606e+02
std,2.178923e+10,6.532366e+07,1.358045e+05,5.143103e+03,7.600104e+03
min,1.000000e+00,1.118030e+09,1.536497e+09,-1.118630e+05,-9.108000e+03
25%,2.268322e+07,1.381743e+09,1.536595e+09,0.000000e+00,0.000000e+00
50%,4.066391e+07,1.434715e+09,1.536741e+09,0.000000e+00,0.000000e+00
75%,5.878004e+07,1.466821e+09,1.536861e+09,0.000000e+00,0.000000e+00
max,1.644962e+11,1.537306e+09,1.537391e+09,1.307661e+07,2.411343e+07


## This task, Pandas takes 4 minutes and 42.0 seconds using NumPy backend, 12.0 seconds using PyArrow backend, and 6.7 seconds to describe the file, 
## while Polars with lazily, fully reading and describing needs 3.1 micro-seconds, 10.6 seconds and 3.5 seconds, respectively.
# ==> Polars wins Pandas in the reading task using *feather format

# In summary, Polars wins all experiments in reading and describing a large data using *csv, *parquet, and *feather formats.
## Pandas with PyArrow backend to read *parquet and *feather is much faster than that of NumPy backend, while Pandas using either PyArrow or NumPy engines does not improve performance to read a large *csv file
# Congratulations Polars (written by Rust and Arrow2)!!!!